In [6]:
import numpy as np
from voxcell import VoxelData
from voxcell.nexus.voxelbrain import Atlas
from utils import *
import morphio
import neurom as nm
from voxcell.math_utils import voxel_intersection
import os
from utils import idx_to_voxel_id
import pandas as pd

In [2]:
atlas_dir = 'data/atlas/25um/'
hierarchy_path = f'{atlas_dir}/hierarchy.json'
sample_morph = 'data/morphologies/swc30/AA0998.swc'

atlas = Atlas.open(atlas_dir)
region_map = atlas.load_region_map()
mask = atlas.get_region_mask('CA1')
vd = VoxelData.load_nrrd(f'{atlas_dir}/brain_regions.nrrd')

In [3]:
gray_matter_nonlayer_obj = ['FF', 'DN', 'SG', 'RT', 'CLI', 'LGv', 'SubG', 'PMd', 'POL', 'PAG', 'PCN', 'LSr', 'IAD', 'Eth', 'FOTU', 'SF', 'VTA', 'ANcr2', 'HY', 'TTv', 'LGd-sh', 'PH', 'SIM', 'DEC', 'VMPO', 'PAS', 'BLAp', 'IMD', 'PAR', 'ECU', 'AVP', 'FS', 'OLF', 'ANcr1', 'LGd-ip', 'IO', 'TTd', 'VCO', 'ACB', 'IPRL', 'RR', 'RE', 'CEAc', 'PVH', 'SPFm', 'PAA', 'LPO', 'LSc', 'PVi', 'GR', 'KF', 'CUN', 'FC', 'PN', 'MMp', 'RCH', 'PMv', 'Pa4', 'SPIV', 'BLAa', 'VPMpc', 'Pa5', 'LIN', 'PYR', 'MA3', 'PO', 'VLPO', 'MB', 'SUT', 'VII', 'BMAp', 'AV', 'DMH', 'PRE', 'PSV', 'RPO', 'IPN', 'SCO', 'SAG', 'VI', 'PVpo', 'MT', 'LP', 'SUB', 'MA', 'POST', 'MV', 'Acs5', 'x', 'APr', 'PPY', 'LAV', 'PPN', 'IPL', 'NTB', 'CENT2', 'PIL', 'CENT3', 'PRP', 'PST', 'AVPV', 'ZI', 'TU', 'ACVII', 'SLC', 'y', 'AT', 'SUV', 'STR', 'MS', 'CUL4, 5', 'Su3', 'MD', 'AP', 'BMAa', 'ARH', 'SMT', 'VPM', 'SCH', 'GRN', 'PRC', 'SPVC', 'IPI', 'DMX', 'VPL', 'VPLpc', 'LA', 'ME', 'NI', 'LT', 'TRS', 'LC', 'CL', 'PARN', 'AD', 'MPN', 'MEPO', 'ASO', 'TR', 'VM', 'SLD', 'VeCB', 'GPe', 'NTS', 'III', 'LRNm', 'CTXsp', 'MPO', 'CM', 'ADP', 'DTN', 'STN', 'OT', 'GPi', 'IRN', 'RPF', 'NOT', 'MARN', 'PRM', 'PDTg', 'TH', 'APN', 'MPT', 'ND', 'MH', 'LGd-co', 'SNc', 'Xi', 'COPY', 'IPR', 'SPVO', 'IGL', 'PIR', 'PeF', 'NLL', 'UVU', 'PGRNl', 'MEA', 'MEV', 'CU', 'CB', 'P', 'NR', 'OV', 'LM', 'TRN', 'XII', 'MGm', 'NOD', 'LRNp', 'RPA', 'BAC', 'IPA', 'PPT', 'LING', 'B', 'CP', 'SUM', 'AHN', 'LH', 'NB', 'MY', 'SOCl', 'BA', 'SNr', 'DCO', 'PSTN', 'PVHd', 'IPC', 'SPVI', 'PFL', 'SBPV', 'ISN', 'MRN', 'V', 'CS', 'SOCm', 'OP', 'IntG', 'AON', 'DP', 'PB', 'CEAl', 'CA1', 'PVp', 'NPC', 'POR', 'INC', 'PCG', 'ICe', 'BST', 'IAM', 'MOB', 'HATA', 'CEAm', 'PT', 'SO', 'P5', 'CLA', 'FN', 'IV', 'IA', 'I5', 'VMH', 'EW', 'FL', 'IPDM', 'PA', 'CA2', 'HPF', 'PRNr', 'PAL', 'CA3', 'SGN', 'RH', 'IPDL', 'DR', 'IF', 'RL', 'SH', 'PS', 'PD', 'COApm', 'MMm', 'MMme', 'PVa', 'LHA', 'PoT', 'ICB', 'COAa', 'ProS', 'ICc', 'LDT', 'MMl', 'COApl', 'SFO', 'VTN', 'PR', 'SI', 'RM', 'IG', 'IP', 'PP', 'RO', 'PG', 'SPFp', 'PC5', 'PBG', 'PRNc', 'NDB', 'PVT', 'PF', 'RN', 'AAA', 'VAL', 'DT', 'SPA']

In [4]:
morphs_dir = 'data/morphologies/swc30/'

In [55]:
morph_dict = {}
for curmorphdir in os.listdir(morphs_dir):
    if not curmorphdir.endswith('.swc'):
        continue

    try: 
        m = nm.load_morphology(f'{morphs_dir}/{curmorphdir}')
        soma_pos = m.soma.points[:,:-1]
        soma_idx = vd.positions_to_indices(soma_pos)
        cur_soma_acronym = region_map.get(idx_to_voxel_id(vd,*soma_idx[0]), "acronym") 
        morph_dict[curmorphdir] = cur_soma_acronym
    except Exception as e:
        print(f'{curmorphdir}: {str(e)}')
        morph_dict[curmorphdir] = np.nan

AA0977.swc: Out of bounds position
AA0952.swc: Out of bounds position
AA0964.swc: Out of bounds position
AA0968.swc: Out of bounds position


In [56]:
df = pd.DataFrame(morph_dict,index=[0]).T
df.columns = ['acronym']

In [57]:
df

,acronym
AA0719.swc,VM
AA0253.swc,dhc
AA0333.swc,FRP6a
AA0961.swc,CENT3
AA0515.swc,PGRNd
...,...
AA0710.swc,PRE
AA0565.swc,dhc
AA0921.swc,VAL
AA0938.swc,PSTN


In [58]:
df.to_csv('morph_regions.csv')

In [59]:
!pwd

/gpfs/bbp.cscs.ch/project/proj142/home/kurban/MMN_Project
